## Vectors

In [1]:
import numpy as np #For arrays
import pickle
import pandas as pd #Gives us DataFrames
import scipy
import gensim
import sklearn.metrics.pairwise #For cosine similarity
import sklearn.manifold #For T-SNE
import sklearn.decomposition #For PCA
import csv
import os #For looking through files
import os.path #For managing file paths
from gensim import models
from gensim.models import Word2Vec

In [5]:
model = Word2Vec.load("word2vec-retrained.model")

In [6]:
print(model.wv.most_similar("engineer"))
print(model.wv.most_similar("painting"))

[('mechanic', 0.6486852765083313), ('engineers', 0.6475141644477844), ('engineering', 0.6428036093711853), ('electrician', 0.6359635591506958), ('programmer', 0.5945500135421753), ('scientist', 0.5807376503944397), ('kups', 0.5781391859054565), ('mechanical', 0.5700127482414246), ('welder', 0.568080723285675), ('happold', 0.5563541650772095)]
[('paintings', 0.8333608508110046), ('artwork', 0.7835107445716858), ('watercolour', 0.7832376956939697), ('painter', 0.7391495108604431), ('portrait', 0.7320064902305603), ('painted', 0.7230610847473145), ('sculpture', 0.7214315533638), ('picasso', 0.7180144786834717), ('watercolours', 0.7133046388626099), ('artworks', 0.7114920020103455)]


In [7]:
def normalize(vector):
    normalized_vector = vector / np.linalg.norm(vector)
    return normalized_vector

def dimension(model, positives, negatives):
    diff = sum([normalize(model.wv[x]) for x in positives]) - sum([normalize(model.wv[y]) for y in negatives])
    return diff

In [8]:
Gender = dimension(model, ['man','him','he', 'male', 'boy'], ['woman', 'her', 'she', 'female', 'girl'])

In [9]:
category_image_parity = {}

In [10]:
category_gender_dimension = {}

In [11]:
missed = []

In [12]:
with open('category_list.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        category, parity = row
        try:
            if len(category.split(' ')) == 2:
                word_1, word_2 = category.split(' ')
                vector = (model.wv[word_1] + model.wv[word_2]) / 2
                category_gender_dimension[category] = sklearn.metrics.pairwise.cosine_similarity(vector.reshape(1,-1), 
                                                                                                 Gender.reshape(1,-1))[0][0]       
            else:
                category_gender_dimension[category] = sklearn.metrics.pairwise.cosine_similarity(model.wv[category].reshape(1,-1), 
                                                                                                 Gender.reshape(1,-1))[0][0]
            category_image_parity[category] = float(parity)

        except KeyError:
            missed.append(category)
            continue
            

In [13]:
cd_out=pd.DataFrame.from_dict(category_gender_dimension.items())

In [18]:
cd_out.columns = ['Social.Category', 'retrained-word2vec-300']

In [19]:
cd_out

,Social.Category,retrained-word2vec-300
0,abandoned infant,-0.039072
1,abandoned person,0.020258
2,able seaman,0.073436
3,abolitionist,-0.099167
4,abortionist,-0.182039
...,...,...
3048,young woman,-0.160440
3049,youngster,0.075502
3050,zen buddhist,-0.009643
3051,zionist,0.020149


In [20]:
savepath=""
cd_out.to_csv(savepath + 'retrained_word2vec_cat_data.csv', index=False)